In [1]:
# import package yang akan digunakan
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
# membaca dataset file csv
df = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")
df

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,The Irishman,2019,18+,7.8/10,98/100,1,0,0,0,0,Martin Scorsese,"Biography,Crime,Drama",United States,"English,Italian,Latin,Spanish,German",209.0
1,1,2,Dangal,2016,7+,8.4/10,97/100,1,0,0,0,0,Nitesh Tiwari,"Action,Biography,Drama,Sport","India,United States,United Kingdom,Australia,K...","Hindi,English",161.0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,9.0/10,95/100,1,0,0,0,0,"Alastair Fothergill,Jonathan Hughes,Keith Scholey","Documentary,Biography",United Kingdom,English,83.0
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,8.1/10,94/100,1,0,0,0,0,Ashutosh Gowariker,"Drama,Musical,Sport","India,United Kingdom","Hindi,English",224.0
4,4,5,Roma,2018,18+,7.7/10,94/100,1,0,0,0,0,NaN,"Action,Drama,History,Romance,War","United Kingdom,United States",English,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9510,9510,9511,Most Wanted Sharks,2020,NaN,NaN,14/100,0,0,0,1,0,NaN,"Crime,Reality-TV",United States,"Greek,English",NaN
9511,9511,9512,Doc McStuffins: The Doc Is In,2020,NaN,NaN,13/100,0,0,0,1,0,Chris Anthony Hamilton,Animation,United States,English,23.0
9512,9512,9513,Ultimate Viking Sword,2019,NaN,NaN,13/100,0,0,0,1,0,NaN,NaN,United States,NaN,NaN
9513,9513,9514,Hunt for the Abominable Snowman,2011,NaN,NaN,10/100,0,0,0,1,0,Dan Oliver,"Drama,History",NaN,NaN,NaN


In [3]:
# menetapkan 'genre' sebagai kolom yang nantinya akan diterapkan association rules
df = df[["Genres"]]
# menghapus data genres yang bernilai kosong atau NaN
df = df.drop(df.loc[df["Genres"].isna()].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9399 entries, 0 to 9514
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genres  9399 non-null   object
dtypes: object(1)
memory usage: 146.9+ KB


In [4]:
# membuat data frame baru yang berisi genre
df_genres = df["Genres"].apply(lambda x: x.split(","))
df_genres.head()

0                 [Biography, Crime, Drama]
1         [Action, Biography, Drama, Sport]
2                  [Documentary, Biography]
3                   [Drama, Musical, Sport]
4    [Action, Drama, History, Romance, War]
Name: Genres, dtype: object

In [5]:
# ubah data frame genre menjadi list
list_data = df_genres.tolist()

In [6]:
te = TransactionEncoder()
df_genres_2 = te.fit(list_data).transform(list_data)
df_genres_2 = pd.DataFrame(df_genres_2, columns=te.columns_)
df_genres_2.head()

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
4,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False


In [13]:
# Algoritma Apriori
df_genres_apriori = apriori(df_genres_2, min_support=0.1, use_colnames=True)
df_genres_apriori

,support,itemsets
0,0.169380,(Action)
1,0.128843,(Adventure)
2,0.364188,(Comedy)
3,0.119268,(Crime)
4,0.124162,(Documentary)
5,0.446856,(Drama)
6,0.146611,(Family)
7,0.172997,(Romance)
8,0.200234,(Thriller)
9,0.114906,"(Comedy, Drama)"


In [14]:
df_rules = association_rules(df_genres_apriori, metric="lift", min_threshold=1)
df_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Romance),(Drama),0.172997,0.446856,0.112459,0.650062,1.454745,0.035154,1.580689
1,(Drama),(Romance),0.446856,0.172997,0.112459,0.251667,1.454745,0.035154,1.105126
2,(Thriller),(Drama),0.200234,0.446856,0.105011,0.524442,1.173626,0.015535,1.163147
3,(Drama),(Thriller),0.446856,0.200234,0.105011,0.235000,1.173626,0.015535,1.045446
